# IBM Data Science specialization capstone project

## Introduction

The problem I would like to solve is to try to predict possible amount of visitors to newly opened chain restaurat depending on the location of this restaurant. This would help stakeholder to decide, where to place new restaurant in the chain to get the most visitors flow.
The idea is that the closer restaurant is to the city center or major transport hub, like aiport or train station, or some other place of attraction -- the more there will be visitors.<br>
If we are talking about a chain fast food restaurat - we can assume that most of them should be very similar, because of franchise conditions. Also people who go to these restaurants mostly already have some expectations based on their previous experience, therefore they are not that biased. So we can assume that amount of visitors to the chain fast food restaurat is some average constant part of the total amount of visitors in all restaurants in that particular area.<br>
### Plan for the project:
1. Get the visit amount for each fast food restaurant in one chain in the city
2. Normalize these values
3. Plot those restaurants on the map of the city with normalized values beside the dots, so that we can visibly examine and see any relationship between the visitor amount ant the restaurant location.
3. Determine one new place on the map where possible new restaurant will be located
4. Predict normalized amount of visitors for the place.
### Data that will be used
I would need to find amount of people who visited one particular restaurat. Then repeat this process for each restaurant in this chain for the selected city. After that I can build a model that predicts the amount of visitors.<br>
After examining Foursquare API documentation, I've decided to use a request that is called "Get Users Who Liked a Venue". At the beginning I wanted to use premium request "Get details of a Venue". It should return me statistics on the total amount of checkins to the venue, bun unfortunately, after some testing the returned json did not have the needed information.<br>

Next idea is to use the amount of total likes for the venue. It is not as accurate as amount of checkins, but might be enough for our task. The more popular is the venue the more likes it will get. More popular venues are visited more. And if we assume that all chain restaurants looks the same and food should be the same, therefore more likes would be in the place which has better location.<br>

As a backup plan, if amount of likes is not accurate enough, I will use the premium request for the venue statistics and use 'ratingSignals' attibute, which is the total amount of people who rated this place on a scale from 1 to 10. We are not interested in the rating itself, but in the amount of people who rated it.<br>

To find all restaurants in the city we will use search requests with a filter on the name that will be the name of the chain. We will use 'venue id', 'longitude' and 'latitude' for each restaurant. We will use coordinate to build a map and 'venue id' is needed to get the amount of 'likes' or 'ratingSignals'.